# Trabajo Práctico N°2

In [2]:
%%writefile README.md

ALLENDE, Lucila Abigail
SERRUYA ALOISI, Luciano Sebastián
TOLEDO MARGALEF, Pablo Adrián

Writing README.md


# Ejercicio 1
#### Cree 3 hilos. El main deberá hacer join sobre el primer hilo. El primer hilo
#### deberá esperar que se cumpla la condición de que un contador administra-
#### do por el segundo hilo llegue a 10. El segundo hilo solo podrá incrementar
#### en uno su contador cuando el tercer hilo, que tiene a su vez un contador
#### interno, lleuge a 10. El tercer hilo tendrá llamadas a la función usleep a
#### con valores aleatorios entre 100mS y 500ms (recuerde que debe multipli-
#### car por 100 para llevar de mS a uS, necesario para la llamada a la función
#### usleep).

In [4]:
%%writefile ej1.c
#include<stdio.h>
#include<stdlib.h>
#include<unistd.h>
#include<pthread.h>

#define MAX 3
#define ESPERA 500

pthread_t hilos[MAX];
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t cond= PTHREAD_COND_INITIALIZER;

int contadorSegundoHilo = 0;


int esperaAleatoria(){
    return (((rand()%4)+1) * 100) * 1000;
}

void *tercerHilo(void *arg){
    
    
    int i;

    for(i = 1; i <= 100; i++){
        
        if (!(i%10)){
            /*
                Si el contador de este hilo llego a diez
                despierta al segundo hilo 
            */
            pthread_cond_signal(&cond);
        }
        
        printf(" %d %d\n", contadorSegundoHilo, i%10);
        usleep(esperaAleatoria());
    }
    
   pthread_exit((void *)0);
}

void *segundoHilo(void *contador){

    pthread_create(&hilos[2], NULL, &tercerHilo, NULL);
    while(contadorSegundoHilo<10){
        
        pthread_mutex_lock(&mutex);

        //esperamos contador interno del tercer hilo
        pthread_cond_wait(&cond,&mutex);
        //incrementamos
        contadorSegundoHilo++;

        pthread_mutex_unlock(&mutex);
        
    }
    
    pthread_exit((void *)0);
    
}

void *primerHilo(void *arg){
    int retval = pthread_create(&hilos[1], NULL, &segundoHilo,NULL);
    if(retval != 0)
        exit(1);
    
    //nos quedamos esperando al segundo hilo
    while(contadorSegundoHilo < 10);
    
    pthread_exit((void *)0);
}





int main(void){
    
    //generacion de random seed
    srand(getpid());
    
    //primer hilo
    int retval = pthread_create(&hilos[0], NULL, &primerHilo,NULL);
    if(retval != 0)
        exit(1);
    
    //joineamos al primer hilo
    pthread_join(hilos[0],NULL);
    
    return 0;
}


Overwriting ej1.c


In [5]:
!gcc -pthread ej1.c -o ej1


In [6]:
!./ej1

 0 1
 0 2
 0 3
 0 4
 0 5
 0 6
 0 7
 0 8
 0 9
 0 0
 1 1
 1 2
 1 3
 1 4
 1 5
 1 6
 1 7
 1 8
 1 9
 1 0
 2 1
 2 2
 2 3
 2 4
 2 5
 2 6
 2 7
 2 8
 2 9
 2 0
 3 1
 3 2
 3 3
 3 4
 3 5
 3 6
 3 7
 3 8
 3 9
 3 0
 4 1
 4 2
 4 3
 4 4
 4 5
 4 6
 4 7
 4 8
 4 9
 4 0
 5 1
 5 2
 5 3
 5 4
 5 5
 5 6
 5 7
 5 8
 5 9
 5 0
 6 1
 6 2
 6 3
 6 4
 6 5
 6 6
 6 7
 6 8
 6 9
 6 0
 7 1
 7 2
 7 3
 7 4
 7 5
 7 6
 7 7
 7 8
 7 9
 7 0
 8 1
 8 2
 8 3
 8 4
 8 5
 8 6
 8 7
 8 8
 8 9
 8 0
 9 1
 9 2
 9 3
 9 4
 9 5
 9 6
 9 7
 9 8
 9 9
 9 0


# Ejercicio 2
#### Cree 10 hilos que atiendan a una misma variable de condición. Despierte a
#### todos los hilos con la una llamada a bilblioteca (averigue cual es). Imprima
#### el instante en el que se despiertan los hilos. Genere una estrategia similar
#### con semáforos. ¿Puede realizar la misma acción?

### (Implementación con mutex)

In [15]:
%%writefile ej2-mutex.c
#include<stdio.h>
#include<stdlib.h>
#include<unistd.h>
#include<pthread.h>

#define MAX 10
#define ESPERA 50000

pthread_t hilos[MAX];
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t cond= PTHREAD_COND_INITIALIZER;


void *meReBloqueo(void *arg){
    int i = *(int *)arg;
    printf("trato bloquear, hilo %d\n",i);
    pthread_mutex_lock(&mutex);
    //al desbloquearse el mutex en el main, este hilo se despierta
    printf("Me desperté, hilo %d\n", i);
    pthread_mutex_unlock(&mutex);
    
}


int main(void){
    pthread_mutex_lock(&mutex);
    
    int i; 
    for(i = 0; i < MAX; i++){
        //creamos hilos
        int retval = pthread_create(&hilos[i], NULL, &meReBloqueo,(void *)&i);
        if(retval != 0)
            exit(1);
        usleep(ESPERA);
    }
    
    //"despertamos" todos los hilos
    pthread_mutex_unlock(&mutex);
    
    for(i = 0; i < MAX; i++){
        //joineamos todos los hilos
        pthread_join(hilos[i],NULL);    
    }
    return 0;
}


Overwriting ej2-mutex.c


In [16]:
!gcc -pthread ej2-mutex.c -o ej2-mutex

In [17]:
!./ej2-mutex

trato bloquear, hilo 0
trato bloquear, hilo 1
trato bloquear, hilo 2
trato bloquear, hilo 3
trato bloquear, hilo 4
trato bloquear, hilo 5
trato bloquear, hilo 6
trato bloquear, hilo 7
trato bloquear, hilo 8
trato bloquear, hilo 9
Me desperté, hilo 0
Me desperté, hilo 1
Me desperté, hilo 2
Me desperté, hilo 3
Me desperté, hilo 4
Me desperté, hilo 5
Me desperté, hilo 6
Me desperté, hilo 7
Me desperté, hilo 8
Me desperté, hilo 9


### (Implementación con condiciones)

In [18]:
%%writefile ej2-condicion.c
#include<stdio.h>
#include<stdlib.h>
#include<unistd.h>
#include<pthread.h>

#define MAX 10
#define ESPERA 50000

pthread_t hilos[MAX];
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t cond= PTHREAD_COND_INITIALIZER;


void *esperoCondicion(void *arg){
    int i = *(int *)arg;
    pthread_mutex_lock(&mutex);
    
    //espera la condicion
    pthread_cond_wait(&cond,&mutex);
    
    
    printf("Me desperté, hilo %d\n", i);
    
    pthread_mutex_unlock(&mutex);
    pthread_exit((void *)0);
}


int main(void){
    
    int i; 
    for(i = 0; i < MAX; i++){
        //creamos hilos
        int retval = pthread_create(&hilos[i], NULL, &esperoCondicion,(void *)&i);
        if(retval != 0)
            exit(1);
        usleep(50000);
    }
    
    
    pthread_cond_broadcast(&cond);
    
    for(i = 0; i < MAX; i++){
        //joineamos todos los hilos
        pthread_join(hilos[i],NULL);    
    }
    return 0;
}

Overwriting ej2-condicion.c


In [19]:
!gcc -pthread ej2-condicion.c -o ej2-condicion

In [20]:
!./ej2-condicion

Me desperté, hilo 0
Me desperté, hilo 1
Me desperté, hilo 2
Me desperté, hilo 3
Me desperté, hilo 4
Me desperté, hilo 5
Me desperté, hilo 6
Me desperté, hilo 7
Me desperté, hilo 8
Me desperté, hilo 9


# Ejercicio 3

#### Cree un programa que cree una pila concurrente:
##### Existe un conjunto de hilos que se determina al momento de ejecutar el
##### programa que hacen PUSH sobre la pila.
##### Existe un conjunto de hilos que se determina al momento de ejecutar el
##### programa que hacen POP sobre la pila. Si la pila está vacía, se bloquean,
##### pero ni bien se hace PUSH, se despiertan.

In [21]:
%%writefile ej3.c
#include<stdio.h>
#include<pthread.h>
#include<unistd.h>
#include<stdlib.h>

#define MAX_PILA            100
#define MAX_HILOS           10
#define CORTE_ITERACIONES   10

#define ESPERA              50000

pthread_t hilosPush[MAX_HILOS];
pthread_t hilosPop[MAX_HILOS];

typedef struct {
    int tope;
    pthread_cond_t llena, vacia;
    pthread_mutex_t mutex;
    int contenido[MAX_PILA];
} pila_t;



/***********************************************************/
/*Tanto en este como en el ej4 (implentación de colas)      /
/*puede pinchar (segmentation fault o stack smashed)        /
/*por lo distintos valores de las constantes, más que nada  /
/*por el CORTE_ITERACIONES.                                 /
/*Ir incrementado o decrementando para ver si pincha o no   /
/***********************************************************/



void crearPila(pila_t *pila){

    pila->tope  = -1;
    pthread_cond_init(&pila->llena,NULL);
    pthread_cond_init(&pila->vacia,NULL);
    pthread_mutex_init(&pila->mutex,NULL);
}


void push(pila_t *pila, int elemento){
    pthread_mutex_lock(&pila->mutex);

    if(pila->tope == (MAX_PILA-1)){
        printf("PILA_LLENA: Espero condicion\n");
        pthread_cond_wait(&pila->llena, &pila->mutex);
    }
    //el tope apunta al elemento visible de la pila
    pila->tope += 1;
    pila->contenido[pila->tope] = elemento;

    pthread_cond_signal(&pila->vacia);

    pthread_mutex_unlock(&pila->mutex);
}


int pop(pila_t *pila){
    pthread_mutex_lock(&pila->mutex);

    if(pila->tope == -1){
        printf("PILA_VACIA: Espero condicion\n");
        pthread_cond_wait(&pila->vacia,&pila->mutex);
    }
    //el tope apunta al elemento visible de la pila
    int valor = pila->contenido[pila->tope];
    pila->tope -= 1;    
    pthread_cond_signal(&pila->llena);
    
    pthread_mutex_unlock(&pila->mutex);

    return valor;
}



int numeroAleatorio(){
    return rand() % 10;
}

//funcion hilosPusheadores
void *hiloPush (void *arg){
    pila_t *p = (pila_t *)arg;
    int corte = 0;
    while(corte < CORTE_ITERACIONES){
        push(p, numeroAleatorio());
        corte++;
    }
    pthread_exit((void *)0);
}

//funcion hilos popeadores
void *hiloPop (void *arg){
    pila_t *p = (pila_t *)arg;
    int corte = 0;
    while(corte < CORTE_ITERACIONES){
        printf("%d\n",pop(p));
        usleep(ESPERA);
        corte++;
    }
    pthread_exit((void *)0);
}


int main(void){
    
    pila_t pila;

    int i,
        retval;

    crearPila(&pila);
    srand(getpid());
    
    
    //hilos pusheadores
    for(i = 0; i < MAX_HILOS ; i++){
        retval = pthread_create(&hilosPush[i], NULL, &hiloPush, (void *)&pila);
        if(retval != 0)
            exit(1);
    }
    
    //hilos popeadores
    for(i = 0; i <  MAX_HILOS ; i++){
        retval = pthread_create(&hilosPop[i], NULL, &hiloPop, (void *)&pila);
        if(retval != 0)
            exit(1);
    }
    
    
    //esperamos hilos popeadores
    for(i = 0; i <  MAX_HILOS ; i++){
        pthread_join(hilosPop[i],NULL);
    }
    
    //esperamos hilos pusheadores
    for(i = 0; i < MAX_HILOS ; i++){
        pthread_join(hilosPush[i],NULL);
    }

    return 0;
}

Overwriting ej3.c


In [22]:
!gcc -pthread ej3.c -o ej3

In [23]:
!./ej3

2
4
3
9
3
2
5
8
1
7
9
7
2
6
3
8
1
8
0
9
3
5
2
2
7
7
4
9
7
4
4
8
0
7
1
5
8
6
9
9
2
4
4
2
3
2
8
9
3
6
5
1
5
3
2
4
9
0
6
3
2
8
0
1
0
6
3
9
0
9
9
0
3
1
2
2
9
3
9
2
8
9
6
0
6
2
4
6
6
6
1
2
7
0
1
0
3
4
0
4


# Ejercicio 4
### Modifique el programa del punto 3 para poder utilizar la pila como cola.

In [24]:
%%writefile ej4.c

#include<stdio.h>
#include<pthread.h>
#include<unistd.h>
#include<stdlib.h>

#define MAX_COLA  			100
#define MAX_HILOS 			10
#define CORTE_ITERACIONES	10

#define ESPERA    			50000


pthread_t hilosEncolar[MAX_HILOS];
pthread_t hilosDesencolar[MAX_HILOS];


typedef struct {
    int final;
    pthread_cond_t llena, vacia;
    pthread_mutex_t mutex;
    int contenido[MAX_COLA];
} cola_t;



int numeroAleatorio(){
    return rand() % 10;
}




void crearCola(cola_t *cola){
	cola->final = -1;
	pthread_cond_init(&cola->llena, NULL);
	pthread_cond_init(&cola->vacia, NULL);
	pthread_mutex_init(&cola->mutex, NULL);

}

void desencolar(cola_t *cola, int *frente){
	pthread_mutex_lock(&cola->mutex);
	
	if(cola->final == -1){
		printf("COLA_VACIA: espero condicion\n");
		pthread_cond_wait(&cola->vacia, &cola->mutex);
	}
	//recuperamos elemento del frente
	*frente = cola->contenido[0];
	
	//Realizamos los corrimientos correspondientes
	int i;
	for(i = 1; i <= cola->final; i++){
		cola->contenido[i-1] = cola->contenido[i];
	}
	cola->final -= 1;
	
	pthread_cond_signal(&cola->llena);
	pthread_mutex_unlock(&cola->mutex);
	
}


void encolar(cola_t *cola, int valor){
    
	pthread_mutex_lock(&cola->mutex);
	
	if(cola->final == MAX_COLA - 1){
		printf("COLA_LLENA: espero condicion\n");
		pthread_cond_wait(&cola->llena, &cola->mutex);
	}
	//insertamos el valor pasado por parametro en la ultima posicion de la cola
	cola->final += 1;
	cola->contenido[cola->final] = valor;
		
	pthread_cond_signal(&cola->vacia);
	pthread_mutex_unlock(&cola->mutex);
    
}


void *funcionHiloDesencola(void *cola){
	cola_t *q = (cola_t *)cola;
	int frente,
		corte = 0;
	while(corte < CORTE_ITERACIONES){
		desencolar(q, &frente);
		printf("%d\n", frente);
		usleep(ESPERA);
		corte++;
	}
	
	pthread_exit((void *)0);

}


void *funcionHiloEncola(void *cola){
	cola_t *q = (cola_t*)cola;
	int corte = 0;
	while(corte < CORTE_ITERACIONES){
		encolar(q, numeroAleatorio());
		corte++;
	}
	
	pthread_exit((void *)0);
}


int main(void){
    
    cola_t q;
    
    int i,
        retval;    
    
    crearCola(&q);
    srand(getpid());
    
    //hilos encoladores
    for(i = 0; i < MAX_HILOS; i++){
        retval = pthread_create(&hilosEncolar[i], NULL, &funcionHiloEncola, (void *)&q);
        if(retval != 0)
            exit(1);
    }
    
    //hilos desencoladores
    for(i = 0; i < MAX_HILOS; i++){
        retval = pthread_create(&hilosDesencolar[i], NULL, &funcionHiloDesencola, (void *)&q);
        if(retval != 0)
            exit(1);
    }
    
    //esperamos hilos encoladores
    for(i = 0; i < MAX_HILOS; i++){
        pthread_join(hilosEncolar[i], NULL);
    }


    //esperamos hilos desencoladores
    for(i = 0; i < MAX_HILOS; i++){
        pthread_join(hilosDesencolar[i], NULL);
    }
    
    return 0;
}

Overwriting ej4.c


In [25]:
!gcc -pthread ej4.c -o ej4

In [26]:
!./ej4

8
0
3
4
2
0
6
1
7
0
7
5
0
9
6
5
4
1
5
7
2
6
8
6
3
0
9
5
3
7
2
6
7
2
5
2
5
2
3
4
4
4
2
5
9
3
0
0
5
0
3
5
3
9
8
6
1
9
1
1
8
3
8
0
5
3
0
8
1
3
9
6
1
6
4
2
6
5
4
0
6
1
5
4
1
6
7
7
4
6
5
5
0
8
0
6
5
0
3
1


# Ejercicio 5
### Escriba un programa que genere N hilos.
#### Cada hilo solicita un número a una función. Esta función solo puede dar
#### N/2 números. Es decir, 2 hilos tendrán el mismo número.
#### Cada hilo debe encontrar a su par y debe realizar un trabajo por un
#### período de tiempo aleatorio.
#### Una vez que termine su operación, deberá avisar que su número está libre.

In [3]:
%%writefile ej5.c

#include<stdio.h>
#include<pthread.h>
#include<unistd.h>
#include<stdlib.h>

#define MAX_HILOS	10
#define ESPERA 		50000

typedef struct 
{
	unsigned char 	valor;
	//unsigned int 	tomado: 2;
	pthread_mutex_t mutex;
	pthread_cond_t 	esperoPar;
} numero_t;


numero_t arreglo[MAX_HILOS/2];
pthread_t hilos[MAX_HILOS];


int numeroAleatorio(int rango){
	return ((rand()) % rango);
}

void arregloAleatorio(){
	int i;
	for(i = 0; i < (MAX_HILOS/2); i++){
		arreglo[i].valor 	= (unsigned char)numeroAleatorio(100);
		//arreglo[i].tomado 	= 0;
		pthread_mutex_init(&arreglo[i].mutex, NULL);
		pthread_cond_init(&arreglo[i].esperoPar, NULL);
	}
}


void *buscoHiloPar(void *args){
	
	int numeroHilo = *(int *)args;
	int posicion = numeroHilo % (MAX_HILOS / 2);
	printf("Hilo %d tiene número %d.\n", numeroHilo, arreglo[posicion].valor);


	/*Enviamos señal. Si es el primer hilo que tiene el número, la señal se pierde.
	Si es el par el que envía la señal, la recibe el primer hilo y sale de la espera*/
	pthread_cond_signal(&arreglo[posicion].esperoPar);

	/*Lockeamos mutex. El primer hilo espera señal del par 
	y el par espera la respuesta del primero*/
	pthread_mutex_lock(&arreglo[posicion].mutex);
	pthread_cond_wait(&arreglo[posicion].esperoPar, &arreglo[posicion].mutex);

	/*Unlockeamos mutex para salir de zona crítica y despertar algún hilo dormido.
	Primer hilo envía respuesta al par*/
	pthread_mutex_unlock(&arreglo[posicion].mutex);
	pthread_cond_signal(&arreglo[posicion].esperoPar);

	printf("El número %d está libre (Hilo %d)\n", arreglo[posicion].valor, numeroHilo);	

	pthread_exit((void *) 0);
}


int main (void){

	srand(getpid());
	arregloAleatorio();

	int i;
	for(i = 0; i < MAX_HILOS; i++){
		pthread_create(&hilos[i], NULL, &buscoHiloPar, (void *)&i);
		usleep(ESPERA * 5);
	}

	for(i = 0; i < MAX_HILOS; i++){
		pthread_join(hilos[i], NULL);
	}


	return 0;
}

Overwriting ej5.c


In [4]:
!gcc -pthread ej5.c -o ej5

In [5]:
!./ej5

Hilo 0 tiene número 40.
Hilo 1 tiene número 51.
Hilo 2 tiene número 4.
Hilo 3 tiene número 16.
Hilo 4 tiene número 3.
Hilo 5 tiene número 40.
El número 40 está libre (Hilo 0)
El número 40 está libre (Hilo 5)
Hilo 6 tiene número 51.
El número 51 está libre (Hilo 1)
El número 51 está libre (Hilo 6)
Hilo 7 tiene número 4.
El número 4 está libre (Hilo 2)
El número 4 está libre (Hilo 7)
Hilo 8 tiene número 16.
El número 16 está libre (Hilo 3)
El número 16 está libre (Hilo 8)
Hilo 9 tiene número 3.
El número 3 está libre (Hilo 4)
El número 3 está libre (Hilo 9)


### Ejercicio 5, otra implementación
#### Menos estática que la anterior

In [7]:
%%writefile ej51.c
#include<stdio.h>
#include<pthread.h>
#include<unistd.h>
#include<stdlib.h>

#define MAX_HILOS	30
#define ESPERA 		500000
#define RANGO_VALORES 10



/***************************************************************************
/* Así como está ahora funciona todo bien,
/* el tema es que como no reinicializamos la bandera de tomado
/* no estaríamos liberando el número, que es lo que nos pide el ejericicio.
***************************************************************************/



typedef struct 
{
	unsigned char 	valor;
	unsigned char 	tomado;
    //0 = valor libre
    //1 = valor asignado a un solo hilo
    //2 = valor asignado a dos hilos
	pthread_mutex_t mutex;
	pthread_cond_t 	esperoPar;
} numero_t;


numero_t arreglo[MAX_HILOS/2];
pthread_t hilos[MAX_HILOS];


int numeroAleatorio(int rango){
	return ((rand()) % rango);
}

void arregloAleatorio(){
	int i;
	for(i = 0; i < (MAX_HILOS/2); i++){
        arreglo[i].valor 	= i;
		arreglo[i].tomado 	= 0;
		pthread_mutex_init(&arreglo[i].mutex, NULL);
		pthread_cond_init(&arreglo[i].esperoPar, NULL);
	}
}


void *buscoHiloPar(void *args){
	
	int numeroHilo = *(int *)args;
    
    int posicion = numeroAleatorio(MAX_HILOS/2);
    
    //int posicion = 0;

    //Buscamos un elemento del arreglo cuyo valor no esté ocupado
    while(arreglo[posicion].tomado >= 2){
        posicion = numeroAleatorio(MAX_HILOS/2);
    }
    
    arreglo[posicion].tomado++;
    
	printf("Hilo %d tiene número %d.\n", numeroHilo, arreglo[posicion].valor);


	/*Enviamos señal. Si es el primer hilo que tiene el número, la señal se pierde.
	Si es el par el que envía la señal, la recibe el primer hilo y sale de la espera*/
	pthread_cond_signal(&arreglo[posicion].esperoPar);

	/*Lockeamos mutex. El primer hilo espera señal del par 
	y el par espera la respuesta del primero*/
	pthread_mutex_lock(&arreglo[posicion].mutex);
	pthread_cond_wait(&arreglo[posicion].esperoPar, &arreglo[posicion].mutex);

	/*Unlockeamos mutex para salir de zona crítica y despertar algún hilo dormido.
	Primer hilo envía respuesta al par*/
	pthread_mutex_unlock(&arreglo[posicion].mutex);
	pthread_cond_signal(&arreglo[posicion].esperoPar);

	printf("El número %d está libre (Hilo %d)\n", arreglo[posicion].valor, numeroHilo);
    
    //Reinicializamos la bandera que indica que el valor está disponible
    //arreglo[posicion].tomado = 0;
    
	pthread_exit((void *) 0);
}


int main (void){

	srand(getpid());
	arregloAleatorio();

	int i;
	for(i = 0; i < MAX_HILOS; i++){
		pthread_create(&hilos[i], NULL, &buscoHiloPar, (void *)&i);
		usleep(ESPERA);
	}

	for(i = 0; i < MAX_HILOS; i++){
		pthread_join(hilos[i], NULL);
	}


	return 0;
}

Writing ej51.c


In [8]:
!gcc -pthread ej51.c -o ej51

In [9]:
!./ej51

Hilo 0 tiene número 11.
Hilo 1 tiene número 13.
Hilo 2 tiene número 14.
Hilo 3 tiene número 1.
Hilo 4 tiene número 12.
Hilo 5 tiene número 1.
El número 1 está libre (Hilo 3)
El número 1 está libre (Hilo 5)
Hilo 6 tiene número 9.
Hilo 7 tiene número 13.
El número 13 está libre (Hilo 1)
El número 13 está libre (Hilo 7)
Hilo 8 tiene número 6.
Hilo 9 tiene número 14.
El número 14 está libre (Hilo 2)
El número 14 está libre (Hilo 9)
Hilo 10 tiene número 12.
El número 12 está libre (Hilo 4)
El número 12 está libre (Hilo 10)
Hilo 11 tiene número 11.
El número 11 está libre (Hilo 0)
El número 11 está libre (Hilo 11)
Hilo 12 tiene número 4.
Hilo 13 tiene número 5.
Hilo 14 tiene número 7.
Hilo 15 tiene número 0.
Hilo 16 tiene número 9.
El número 9 está libre (Hilo 6)
El número 9 está libre (Hilo 16)
Hilo 17 tiene número 7.
El número 7 está libre (Hilo 14)
El número 7 está libre (Hilo 17)
Hilo 18 tiene número 8.
Hilo 19 tiene número 8.
El número 8 está libre (Hilo 18)
El número 8 está libre (Hilo